# Data exploration

In [3]:
import pandas as pd
import numpy as np
from helpers import count_unique_words
from gensim.models import Word2Vec
import gensim

/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/stefano/anaconda3/envs/tf-gpu/lib/pyt

In [26]:
df = pd.read_pickle("dataframes/full_df_cleaned_test_0_2.pickle")

In [9]:
chi2 = pd.read_pickle("dataframes/chi2_google_words.pickle")

In [24]:
chi2.set_index("word", inplace=True)

In [25]:
chi2

,chi2
word,
follow,21973.469174
naked,21.476155
cheetah,1.080130
goodness,28.993678
sake,108.046105
...,...
worktops,1.000458
suckout,1.000458
shrinkwrap,1.000458


In [5]:
df.head()

,sentence,label
1621312,rt butt swollen #skaterprobz aww boo ! fall ? ...,-1
2306732,foot calf hip back ache .. i'm gettin old can'...,-1
1337824,episode zero stewie hell,-1
801564,lol well time always good ?,1
1950982,omg mum selling soccer aid ticket cry eye,-1


In [6]:
my_value_counts = df.iloc[:].sentence.apply(lambda x: x.split()).explode().value_counts()

In [7]:
print(len(my_value_counts[my_value_counts < 2]))
print(len(my_value_counts))

114478
186356


In [31]:
hashtags_count = 0
hashtags = []
for el in my_value_counts.index:
    if str.startswith(el, "#"):
        hashtags_count += 1
        hashtags.append(el)

In [36]:
my_value_counts[hashtags][my_value_counts<10]

#blondemoment            9
#theelites               9
#61khunter               9
#thankspep               9
#kyuevilquotes           9
                        ..
#weloveshout             1
#sinner                  1
#letshaveahouseparty     1
#foreverdisliked         1
#favoritepartoftheday    1
Name: sentence, Length: 92774, dtype: int64

In [6]:
df_new = df.sentence.apply(lambda x: " ".join([el if my_value_counts[el]>1 else "" for el in x.split()]))

In [7]:
df.sentence = df_new

In [8]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format("models/GoogleNews-vectors-negative300.bin", binary=True)

In [27]:
unique_words = df['sentence'].apply(lambda x: x.split()).explode().unique()
len(unique_words)

186357

In [28]:
%time df_new = df['sentence'].apply(lambda x: " ".join([el if el in w2v_model.wv.vocab else "" for el in x.split()]))

/home/stefano/anaconda3/envs/tf-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


CPU times: user 6.8 s, sys: 0 ns, total: 6.8 s
Wall time: 6.82 s


In [29]:
df_new

1621312             rt butt swollen  aww boo  fall  bruised 
2306732    foot calf hip back ache   gettin old can't roc...
1337824                                   episode zero  hell
801564                            lol well time always good 
1950982            omg mum selling soccer aid ticket cry eye
                                 ...                        
1417173    video league legend ip cheat  patch please wat...
1373765                           finshed chance better  :/ 
790399                         want follow back  lunch break
85724                           know   we're singing chamber
1289016            im soo sorry  grandpa  even  happen ago  
Name: sentence, Length: 500000, dtype: object

In [30]:
df.sentence = df_new
print(df)

                                                  sentence  label
1621312           rt butt swollen  aww boo  fall  bruised      -1
2306732  foot calf hip back ache   gettin old can't roc...     -1
1337824                                 episode zero  hell     -1
801564                          lol well time always good       1
1950982          omg mum selling soccer aid ticket cry eye     -1
...                                                    ...    ...
1417173  video league legend ip cheat  patch please wat...     -1
1373765                         finshed chance better  :/      -1
790399                       want follow back  lunch break      1
85724                         know   we're singing chamber      1
1289016          im soo sorry  grandpa  even  happen ago       -1

[500000 rows x 2 columns]


In [14]:
len(df[df.sentence == '']) 

1939

In [32]:
df_new = df['sentence'].apply(lambda x: " ".join(" ".join(
    [el if
         el in chi2.index and not chi2.loc[el].isna()[0]
     else "" 
     for el in x.split()]).split() )) # Second split is used to remove multiple spaces

In [33]:
df.sentence = df_new

In [35]:
unique_words = df['sentence'].apply(lambda x: x.split()).explode().unique()
len(unique_words)

43637

In [36]:
len(df[df.sentence == '']) 

5376

In [40]:
new_value_counts = df.iloc[:]['sentence'].apply(lambda x: x.split()).explode().value_counts()

In [42]:
new_value_counts[new_value_counts==1]

screensavers    1
implicitly      1
yomp            1
globale         1
antigravity     1
               ..
hepatic         1
samll           1
shedule         1
nannying        1
postmistress    1
Name: sentence, Length: 13095, dtype: int64

In [14]:
df.to_pickle("dataframes/dataframe_train_0_2_google.pickle")